In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from tqdm import tqdm

In [ ]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [ ]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

In [8]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [ ]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print("Train features:", X_train.shape)
print("Train gap:", Y_train.shape)
print("Test features:", X_test.shape)

In [19]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

MemoryError: 

In [16]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

In [ ]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [20]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)

NameError: name 'LR_pred' is not defined

In [9]:
better_regressor = RandomForestRegressor(oob_score = True,n_jobs = 4,random_state =1)

In [23]:
better_regressor.fit(X_train, Y_train)


MemoryError: 

In [32]:
pred = better_regressor.predict(X_test) 

In [33]:
print(RF.score(X_train,Y_train), better_regressor.score(X_train,Y_train))

0.554162269346 0.554162534764


In [13]:
msk = np.random.rand(len(X_train)) < 10000/len(X_train)

In [14]:
kf = KFold(n_splits=5)

x_train = X_train[msk]
y_train = Y_train[msk]

num_trees = 64
val_rates_RF = []
for p in tqdm(range(12)):
    params = (num_trees, p)
    temp_scores = []
    for train_index, val_index in kf.split(x_train):
        X_train_split, X_val_split = x_train[train_index], x_train[val_index]
        y_train_split, y_val_split = y_train[train_index], y_train[val_index]
        est = RandomForestRegressor(oob_score=True, n_estimators=num_trees, n_jobs=-1)
        est.fit(X_train_split, y_train_split)
        temp_scores.append(est.score(X_val_split, y_val_split))
    val_rates_RF.append(np.mean(temp_scores))

best_n = np.argmax(val_rates_RF) + 1

100%|██████████| 12/12 [00:25<00:00,  2.11s/it]


In [15]:
print(best_n)

12


In [ ]:
better_regressor = RandomForestRegressor(oob_score = True, n_estimators = 12, n_jobs = -1)
better_regressor.fit(X_train, Y_train)
pred = better_regressor.predict(X_test) 

In [ ]:
print(better_regressor.score(X_train,Y_train))